In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import collections
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

import requests
from bs4 import BeautifulSoup as bs


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Error loading vader_lexicon: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
/kaggle/input/sf-booking/hotels_test.csv
/kaggle/input/sf-booking/hotels_train.csv
/kaggle/input/sf-booking/submission.csv


In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(2)

In [5]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

hotels = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng,sample,reviewer_score
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102,0,0.0
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443,0,0.0
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,11/26/2015,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,5,15,"[' Business trip ', ' Solo traveler ', ' Twin ...",616 day,41.393192,2.161520,0,0.0
3,Piazza Della Repubblica 17 Central Station 201...,241,10/17/2015,9.1,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,9,"[' Leisure trip ', ' Couple ', ' Ambassador Ju...",656 day,45.479888,9.196298,0,0.0
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,5/16/2016,9.1,Hotel Esther a,United Kingdom,No Negative,0,4687,Lovely hotel v welcoming staff,7,2,"[' Business trip ', ' Solo traveler ', ' Class...",444 day,52.370545,4.888644,0,0.0


In [6]:
# Переводим столбец review_date в временной формат и создаём новый признак - month
hotels['review_date'] = pd.to_datetime(hotels['review_date'])

hotels['month'] = hotels['review_date'].dt.month

In [7]:
hotels['score_without_review_per'] = hotels['additional_number_of_scoring'] / hotels['total_number_of_reviews'] * 100
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng,sample,reviewer_score,month,score_without_review_per
0,Via Senigallia 6 20161 Milan Italy,904,2017-07-21,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102,0,0.0,7,5.422915
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,2016-12-12,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443,0,0.0,12,12.196094
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,2015-11-26,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,5,15,"[' Business trip ', ' Solo traveler ', ' Twin ...",616 day,41.393192,2.161520,0,0.0,11,13.105413
3,Piazza Della Repubblica 17 Central Station 201...,241,2015-10-17,9.1,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,9,"[' Leisure trip ', ' Couple ', ' Ambassador Ju...",656 day,45.479888,9.196298,0,0.0,10,15.618924
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,2016-05-16,9.1,Hotel Esther a,United Kingdom,No Negative,0,4687,Lovely hotel v welcoming staff,7,2,"[' Business trip ', ' Solo traveler ', ' Class...",444 day,52.370545,4.888644,0,0.0,5,17.793898


In [8]:
mask = hotels.loc[:, 'total_number_of_reviews_reviewer_has_given'] == 1
hotels[mask].loc[:, 'reviewer_score'].value_counts().sort_index(ascending=False)

def great_mark(mark):
    if mark == 10:
        return 1
    else:
        return 0
        
def first_mark(mark_count):
    if mark_count == 1:
        return 1
    else:
        return 0
    
hotels.loc[:, 'is_first_mark'] = hotels['total_number_of_reviews_reviewer_has_given'].apply(first_mark)
hotels.loc[:, 'is_great_mark'] = hotels['reviewer_score'].apply(great_mark)

In [8]:
tags_group_list = hotels['tags'].values
tags_list = []
for tag_group in tags_group_list:
    tags = tag_group.split(', ')
    for tag in tags:
        clean_tag = re.sub(r'[\'\[\]]', '', tag)
        tags_list.append(clean_tag)
        
unique_tag = set(tags_list)
len(unique_tag)

2428

In [9]:
# Анализируем отзывы
sent_analyzer = SentimentIntensityAnalyzer()

hotels['rw_neg'] = hotels['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x))
hotels['rw_pos'] = hotels['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x)) 
    
hotels.loc[:,['n_neg', 'n_neu', 'n_pos', 'n_compound']] = list(hotels['rw_neg'].apply(lambda x: [x['neg'], x['neu'], x['pos'], x['compound']]).values)
hotels.loc[:,['p_neg', 'p_neu', 'p_pos', 'p_compound']] = list(hotels['rw_pos'].apply(lambda x: [x['neg'], x['neu'], x['pos'], x['compound']]).values)

# Выведем процентное содержание позитивных и негативных слов
hotels['positive_word_per'] = 100 * hotels['review_total_positive_word_counts'] / (hotels['review_total_negative_word_counts'] + hotels['review_total_positive_word_counts'])
hotels['negative_word_per'] = 100 * hotels['review_total_negative_word_counts'] / (hotels['review_total_negative_word_counts'] + hotels['review_total_positive_word_counts'])
hotels['positive_word_per'].fillna(0,inplace=True)
hotels['negative_word_per'].fillna(0,inplace=True)

In [10]:
# Определяем множество уникальных мест(страна и город)
place = {' '.join(x.split()[-2:]) for x in hotels.loc[:, 'hotel_address'].values}

In [11]:
# Создаём функцию для определения страны
def get_country(address):
    for i in place:
        if i in address:
            if i == 'United Kingdom':
                return i
            else:
                return ''.join(i.split()[-1])

# Создаём функцию для определения города
def get_city(address):
    for i in place:
        if i in address:
            if i == 'United Kingdom':
                return 'London'
            else:
                return ''.join(i.split()[0])
            
hotels['city'] = hotels['hotel_address'].apply(get_city)
#hotels['country'] = hotels['hotel_address'].apply(get_country)

hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: int(''.join(x.split()[0])))

#Создаём категории средних оценок
def score_group(score):
    if score >= 9.5:
        return 'excelent'
    elif 9.0 < score < 9.5:
        return 'high'
    elif score < 9.0 and score >= 8.0:
        return 'good'
    else:
        return 'normal'
    
hotels['hotel_level'] = hotels['average_score'].apply(score_group) 

# Определяем содержит ли отзыв негативные содержание 
# Определяем содержит ли отзыв положительное содержание 
def negative(answer):
    if 'nothing' in answer.lower() or 'none' in answer.lower() or 'n a' in answer.lower() or 'no negative' in answer.lower():
        return 0
    else:
        return 1
    
def positive(answer):
    if 'nothing' in answer.lower() or 'none' in answer.lower() or 'n a' in answer.lower() or 'no positive' in answer.lower():
        return 0
    else:
        return 1
    
hotels['is_positive_review'] = hotels['positive_review'].apply(positive) 
hotels['is_negative_review'] = hotels['negative_review'].apply(negative)

# Определяем тип путешествия
def trip(tag):
    tag_list = tag.split(', ')
    trip_kind = tag_list[0]
    trip_kind = re.findall(r'\w+\s\w+', trip_kind)
    result = ''.join(trip_kind)
    return result
    
hotels['kind_of_trip'] = hotels['tags'].apply(trip)

# Определяем был ли отзыв оставлен с телефона
def mob_device(tag):
    tag_list = tag.split(',')
    if 'submitted' in tag_list[-1].lower():
        return 1
    else:
        return 0
    

hotels['submitted_from_a_mobile_device'] = hotels['tags'].apply(mob_device)

# Определяем вид комнаты 
def kind_of_room(tag):
    try:
        tag_list = tag.split(',')
        trip_kind = tag_list[2]
        trip_kind = re.sub(r'[\']', '', trip_kind)
        trip_kind = re.sub(r'^\s|\s$', '', trip_kind)
        return trip_kind
    except IndexError:
        return None

hotels['kind_of_room'] = hotels['tags'].apply(kind_of_room)


In [13]:
values = {
    'lat':hotels['lat'].mode()[0],
    'lng':hotels['lng'].mode()[0],
    'kind_of_room':hotels['kind_of_room'].mode()[0]
}

hotels.fillna(values, inplace=True)
hotels.isnull().mean()

hotel_address                                 0.0
additional_number_of_scoring                  0.0
review_date                                   0.0
average_score                                 0.0
hotel_name                                    0.0
reviewer_nationality                          0.0
negative_review                               0.0
review_total_negative_word_counts             0.0
total_number_of_reviews                       0.0
positive_review                               0.0
review_total_positive_word_counts             0.0
total_number_of_reviews_reviewer_has_given    0.0
tags                                          0.0
days_since_review                             0.0
lat                                           0.0
lng                                           0.0
sample                                        0.0
reviewer_score                                0.0
month                                         0.0
score_without_review_per                      0.0


In [14]:
# Кодируем объекты типа Object
import category_encoders as ce

encoder = ce.OneHotEncoder(['city', 'kind_of_trip', 'hotel_level'])
type_bin = encoder.fit_transform(hotels.loc[:, ['city', 'kind_of_trip', 'hotel_level']])

df = pd.concat([hotels, type_bin], axis=1)

bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality', 'kind_of_room', 'hotel_name'])
type_bin = bin_encoder.fit_transform(hotels.loc[:, ['reviewer_nationality', 'kind_of_room', 'hotel_name']])
df = pd.concat([df, type_bin], axis=1)

df.drop(['city', 'kind_of_trip', 'hotel_level', 'reviewer_nationality', 'kind_of_room', 'hotel_name'], axis=1, inplace=True)

In [15]:
from sklearn import preprocessing
#инициализируем нормализатор MinMaxScaler
col_names = ['additional_number_of_scoring','total_number_of_reviews', 'days_since_review', 'month', 'review_total_negative_word_counts', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given']
mm_scaler = preprocessing.MinMaxScaler()

# копируем исходный датасет
df_mm = mm_scaler.fit_transform(df.loc[:, ['additional_number_of_scoring','total_number_of_reviews', 'days_since_review', 'month', 'review_total_negative_word_counts', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given']])

df_mm = pd.DataFrame(df_mm, columns=col_names)
df.drop(col_names, axis=1, inplace=True)
df = pd.concat([df, df_mm], axis=1)
df.head()

,hotel_address,review_date,average_score,negative_review,positive_review,tags,lat,lng,sample,reviewer_score,...,hotel_name_8,hotel_name_9,hotel_name_10,additional_number_of_scoring,total_number_of_reviews,days_since_review,month,review_total_negative_word_counts,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given
0,Via Senigallia 6 20161 Milan Italy,2017-07-21,8.1,Would have appreciated a shop in the hotel th...,Hotel was great clean friendly staff free bre...,"[' Leisure trip ', ' Couple ', ' Double Room '...",45.533137,9.171102,0,0.0,...,0,0,1,0.336815,1.000000,0.017808,0.545455,0.127451,0.156962,0.000000
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,2016-12-12,8.6,No tissue paper box was present at the room,No Positive,"[' Leisure trip ', ' Group ', ' Triple Room ',...",52.385649,4.834443,0,0.0,...,0,1,0,0.227900,0.299212,0.320548,1.000000,0.024510,0.000000,0.016949
2,Mallorca 251 Eixample 08008 Barcelona Spain,2015-11-26,8.3,Pillows,Nice welcoming and service,"[' Business trip ', ' Solo traveler ', ' Twin ...",41.393192,2.161520,0,0.0,...,0,1,1,0.016785,0.018524,0.843836,0.909091,0.007353,0.012658,0.039548
3,Piazza Della Repubblica 17 Central Station 201...,2015-10-17,9.1,No Negative,Everything including the nice upgrade The Hot...,"[' Leisure trip ', ' Couple ', ' Ambassador Ju...",45.479888,9.196298,0,0.0,...,1,0,0,0.089519,0.090215,0.898630,0.818182,0.000000,0.068354,0.022599
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,2016-05-16,9.1,No Negative,Lovely hotel v welcoming staff,"[' Business trip ', ' Solo traveler ', ' Class...",52.370545,4.888644,0,0.0,...,1,0,1,0.310705,0.279305,0.608219,0.363636,0.000000,0.017722,0.002825


In [16]:
col_names = ['additional_number_of_scoring','total_number_of_reviews', 'days_since_review', 'month', 'lat', 'lng', 'hotel_address', 'review_date', 'negative_review', 'positive_review', 'tags', 'rw_neg', 'rw_pos']
df.drop(col_names, axis=1, inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 71 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   average_score                               515738 non-null  float64
 1   sample                                      515738 non-null  int64  
 2   reviewer_score                              515738 non-null  float64
 3   score_without_review_per                    515738 non-null  float64
 4   is_first_mark                               515738 non-null  int64  
 5   is_great_mark                               515738 non-null  int64  
 6   n_neg                                       515738 non-null  float64
 7   n_neu                                       515738 non-null  float64
 8   n_pos                                       515738 non-null  float64
 9   n_compound                                  515738 non-null  float64
 

In [18]:
# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [19]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [20]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

((128935, 70), (386803, 70), (386803, 69), (309442, 69), (77361, 69))

In [21]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [22]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [23]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.7min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.4s finished


In [31]:
def MAPE(y_true, y_pred): 
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / np.maximum(np.ones(len(y_true)), np.abs(y_true))))*100

# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MAPE:', MAPE(y_test, y_pred))

MAE: 0.689354293973851
MAPE: 10.350589377259428


In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (15,18)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(70).plot(kind='barh')

In [32]:
test_data.sample(10)

,average_score,reviewer_score,score_without_review_per,is_first_mark,is_great_mark,n_neg,n_neu,n_pos,n_compound,p_neg,...,hotel_name_4,hotel_name_5,hotel_name_6,hotel_name_7,hotel_name_8,hotel_name_9,hotel_name_10,review_total_negative_word_counts,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given
85407,9.1,0.0,13.599182,0,0,1.000,0.000,0.000,-0.7096,0.0,...,1,1,1,0,0,0,1,0.000000,0.022785,0.014124
8815,8.0,0.0,20.591341,0,0,0.688,0.312,0.000,-0.2960,0.0,...,0,1,1,1,1,0,0,0.007353,0.020253,0.144068
23028,9.6,0.0,15.846995,0,0,1.000,0.000,0.000,-0.7096,0.0,...,1,1,0,1,1,0,1,0.000000,0.392405,0.028249
60127,8.7,0.0,24.082314,0,0,1.000,0.000,0.000,-0.7096,0.0,...,1,0,0,0,0,0,0,0.000000,0.043038,0.005650
61426,7.6,0.0,11.950355,0,0,0.000,1.000,0.000,0.0000,0.0,...,0,1,0,0,1,1,0,0.009804,0.030380,0.158192
89341,9.1,0.0,29.141717,0,0,0.099,0.811,0.090,0.1189,0.0,...,0,0,0,0,0,0,0,0.154412,0.060759,0.002825
88113,9.1,0.0,11.309524,1,0,0.000,0.864,0.136,0.0258,0.0,...,1,1,1,0,1,0,1,0.029412,0.017722,0.000000
114380,8.3,0.0,24.948735,0,0,0.000,1.000,0.000,0.0000,0.0,...,0,1,0,0,1,1,0,0.083333,0.035443,0.005650
69101,8.7,0.0,24.497732,0,0,1.000,0.000,0.000,-0.7096,0.0,...,0,1,1,1,0,1,0,0.000000,0.005063,0.011299
52547,8.4,0.0,8.636093,0,0,1.000,0.000,0.000,-0.7096,0.0,...,0,1,1,0,0,0,0,0.000000,0.058228,0.008475


In [33]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [34]:
sample_submission

,reviewer_score,id
0,1,488440
1,10,274649
2,3,374688
3,10,404352
4,9,451596
...,...,...
128930,7,495812
128931,1,315577
128932,3,511968
128933,6,303386


In [35]:
predict_submission = model.predict(test_data)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.8s finished


In [36]:
predict_submission

array([8.42 , 7.597, 7.947, ..., 7.821, 9.222, 7.883])

In [37]:
list(sample_submission)

['reviewer_score', 'id']

In [38]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission_new_try.csv', index=False)
sample_submission.head(10)

,reviewer_score,id
0,8.420,488440
1,7.597,274649
2,7.947,374688
3,9.303,404352
4,9.153,451596
5,8.668,302161
6,7.885,317079
7,7.475,13963
8,8.190,159785
9,7.563,195089
